## 处理A股数据

In [ ]:
import pandas as pd
import pickle
from tqdm import tqdm


In [ ]:

new_data = pickle.load(open(r'/home/xuyang1/PatchTST/our_align_data.lm.V06.pickle/new_data.pkl', 'rb'))

# data = pickle.load(open(r'/home/xuyang1/PatchTST/our_align_data.lm.V06.pickle/share.pkl', 'rb'))
# ret = pickle.load(open(r'/home/xuyang1/PatchTST/ret.pkl', 'rb'))

# ret = ret.reindex(data.index)
# new_data = pd.concat([data.iloc(axis=1)[:-5], ret], axis=1)

In [ ]:
# for col in tqdm(new_data.columns):
#     tmp_df = pd.DataFrame({"left": new_data[col], "right": new_data.iloc(axis=1)[-5]})
#     ic1 = tmp_df.groupby("datetime").apply(lambda df: df["left"].corr(df["right"])).mean()
#     tmp_df = pd.DataFrame({"left": new_data[col], "right": new_data.iloc(axis=1)[-3]})
#     ic2 = tmp_df.groupby("datetime").apply(lambda df: df["left"].corr(df["right"])).mean()
#     if ic1 < 0 and ic2 < 0:
#         new_data[col] *= -1
#         print(col, ic1.mean(), ic2.mean())
    # assert ic1 > 0 == ic2 > 0

#     ic = df.groupby("datetime").apply(lambda df: df["pred"].corr(df["label"]))


# for index, col in enumerate(data.columns):
#     print(index, col)
# for index, col in enumerate(ret.columns):
#     print(index, col)

# for index in range(5):
#     pred = data.iloc[:, -5 + index]
#     label = ret.iloc[:, index]
#     label = label.reindex(pred.index)
#     print(pred.shape, label.shape)
#     df = pd.DataFrame({"pred": pred, "label": label})
#     ic = df.groupby("datetime").apply(lambda df: df["pred"].corr(df["label"]))
#     ric = df.groupby("datetime").apply(lambda df: df["pred"].corr(df["label"], method="spearman"))
#     print(ic.mean(), ric.mean())

In [ ]:

display(new_data.shape)
display(new_data.head())
# pickle.dump(new_data, open(r'/home/xuyang1/PatchTST/our_align_data.lm.V06.pickle/new_data.pkl', 'wb'))

In [ ]:
from pandarallel import pandarallel
pandarallel.initialize()

tqdm.pandas()
def calculate_quintile_ret(df):
    label = df.iloc(axis=1)[-5]
    quintile_ret = pd.Series(index=df.columns)
    for col in df.columns:
        tmp_df = pd.DataFrame({"left": df[col], "right": label}).sort_values("left", ascending=False)
        group1_ret = tmp_df.iloc[:int(len(tmp_df) * 0.2), 1].mean()
        group5_ret = tmp_df.iloc[int(len(tmp_df) * 0.8):, 1].mean()
        quintile_ret[col] = group1_ret - group5_ret

    return quintile_ret

# quintile_ret_data = new_data.groupby("datetime").parallel_apply(calculate_quintile_ret)
# pickle.dump(quintile_ret_data, open(r'/home/xuyang1/PatchTST/our_align_data.lm.V06.pickle/quintile_ret_data.pkl', 'wb'))

In [ ]:
from pandarallel import pandarallel
pandarallel.initialize()

def calculate_top10p_ret(df):
    label = df.iloc(axis=1)[-5]
    top10p_ret = pd.Series(index=df.columns)
    for col in df.columns:
        tmp_df = pd.DataFrame({"left": df[col], "right": label}).sort_values("left", ascending=False)
        top10p_ret[col] = tmp_df.iloc[:int(len(tmp_df) * 0.1), 1].mean()
    return top10p_ret

# top10p_ret_data = new_data.groupby("datetime").parallel_apply(calculate_top10p_ret)
# pickle.dump(top10p_ret_data, open(r'/home/xuyang1/PatchTST/our_align_data.lm.V06.pickle/top10p_ret_data.pkl', 'wb'))

In [ ]:

# top10p_ret_data = pickle.load(open(r'/home/xuyang1/PatchTST/our_align_data.lm.V06.pickle/top10p_ret_data.pkl', 'rb'))
# top10p_ret_data.columns = [col[1] for col in top10p_ret_data.columns]
# import plotly.express as px
# top10p_ret_data_cumsum = top10p_ret_data.cumsum().iloc(axis=1)[::5]
# fig = px.line(top10p_ret_data_cumsum, title="Top 10% cumulative return")
# fig.show()


In [ ]:
quintile_ret_data = pickle.load(open(r'/home/xuyang1/PatchTST/our_align_data.lm.V06.pickle/quintile_ret_data.pkl', 'rb'))
top10p_ret_data = pickle.load(open(r'/home/xuyang1/PatchTST/our_align_data.lm.V06.pickle/top10p_ret_data.pkl', 'rb'))

quintile_ret_data_nolabel = quintile_ret_data.iloc(axis=1)[:-5]
top10p_ret_data_nolabel = top10p_ret_data.iloc(axis=1)[:-5]
quintile_ret_data_nolabel.columns = [col[1] for col in quintile_ret_data_nolabel.columns]
top10p_ret_data_nolabel.columns = [col[1] for col in top10p_ret_data_nolabel.columns]
quintile_ret_data_nolabel.index.names = ["date"]
top10p_ret_data_nolabel.index.names = ["date"]

quintile_ret_data_nolabel.to_csv(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_stocka.csv')
top10p_ret_data_nolabel.to_csv(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_us.csv')


In [ ]:
import os
shape_series = pd.Series()
import plotly.express as px
for name in os.listdir(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/'):
    tmp = pd.read_csv(os.path.join(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/', name))
    shape_series[name] = tmp.shape
    if "stocka" in name:
        tmp = tmp.set_index("date")
        tmp = tmp.rolling(20).mean()
        fig = px.line(tmp.iloc(axis=1)[::10], title=f"{name} Top 10% cumulative return")
        fig.show()
    
# display(shape_series.to_frame("dataset_size"))

## 验证A股结果

In [ ]:
import pandas as pd
import os
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

label_quintile = pd.read_csv(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_stocka.csv', index_col=0)
# label_quintile = pd.DataFrame(pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_stocka_transformed.pkl', 'rb')), index=label_quintile.index, columns=label_quintile.columns)
quintile_scaler = pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_stocka_scaler.pkl', 'rb'))
label_top10p = pd.read_csv(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_stocka.csv', index_col=0)
# label_top10p = pd.DataFrame(pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_stocka_transformed.pkl', 'rb')), index=label_quintile.index, columns=label_quintile.columns)
top10p_scaler = pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_stocka_scaler.pkl', 'rb'))
losses = []
for name in os.listdir(r"/home/xuyang1/PatchTST/PatchTST_supervised/results"):
    model = name.split("_")[4]
    seq_length = name.split("_")[3]
    dataset = name.split("_")[0]
    if dataset == "quintile":
        label = label_quintile
    elif dataset == "top10p":
        label = label_top10p
    pred = np.load(rf"/home/xuyang1/PatchTST/PatchTST_supervised/results/{name}/pred.npy")

    if dataset == "quintile":
        scaler = quintile_scaler
    elif dataset == "top10p":
        scaler = top10p_scaler
    pred = scaler.inverse_transform(np.reshape(pred, (-1, pred.shape[-1]))).reshape(pred.shape)

    # if "quinti" not in dataset or seq_length != "96":
    #     continue

    test_length = pred.shape[0] + pred.shape[1] - 1
    target_label = label.iloc[-test_length:,:]


    # calculate pred to real label loss:
    # loss = 0
    # for i in range(pred.shape[0]):
    #     pred_batch = pred[i]
    #     label_batch = target_label.iloc[i:i+pred.shape[1],:].values
    #     loss += np.mean((pred_batch - label_batch)**2)

    # calculate real return with prediction:
    # 1. use closest pred as real pread
    real_pred = np.concatenate([pred[:-1,0,:], pred[-1,:,:]], axis=0)

    # 2. use mean of pred as real pred
    # pred_middata_np = np.zeros((pred.shape[0], pred.shape[0] + pred.shape[1] - 1, pred.shape[2]))
    # pred_middata_np[:,:,:] = np.nan
    # for i in range(pred.shape[0]):
    #     pred_middata_np[i, i:i+pred.shape[1], :] = pred[i]
    # real_pred = np.nanmean(pred_middata_np, axis=0, keepdims=False)


    # 3. use mean of pred as real pred, but only use the first 20 pred
    # real_pred = pred[:, :20, :].mean(axis=1, keepdims=False)

    # pred_df = pd.DataFrame(real_pred, index=target_label.index[:real_pred.shape[0]], columns=target_label.columns)

    # rolling_20_label = target_label.rolling(20).sum().shift(-20).iloc(axis=0)[:pred.shape[0]]

    # average_ret = rolling_20_label.mean(axis=1).to_frame("average_ret")

    # rolling_20_pred_df = pred_df.rolling(20).mean().shift(-20).iloc(axis=0)[:-20]
    # top50p_pred_ret = rolling_20_label[(pred_df.rank(axis=1, pct=True) > 0.5)].mean(axis=1).to_frame("top50p_pred_ret")
    # top20p_pred_ret = rolling_20_label[(pred_df.rank(axis=1, pct=True) > 0.8)].mean(axis=1).to_frame("top20p_pred_ret")
    # top10p_pred_ret = rolling_20_label[(pred_df.rank(axis=1, pct=True) > 0.9)].mean(axis=1).to_frame("top10p_pred_ret")
    # break

    # show_df = pd.concat([average_ret, top50p_pred_ret, top20p_pred_ret, top10p_pred_ret], axis=1).cumsum(axis=0)
    # show_df.plot()
    # plt.title(f"{dataset} {model} {seq_length}")
    # plt.show()

    # pred_with_input = pd.concat([label.iloc[:-test_length,:], pred_df], axis=0)
    # pred_with_input.columns = [col + "_pred" for col in pred_with_input.columns]

    # fig, axes = plt.subplots(4, 4)
    # fig.set_figheight(30)
    # fig.set_figwidth(30)
    # plt.subplots_adjust(wspace=None, hspace=None)
    # axes = axes.reshape(4, 4)
    # stride = label.shape[-1] // 16
    # for i in range(16):
    #     pred_and_gt = pd.concat([label.iloc(axis=1)[i*stride:i*stride+1], pred_with_input.iloc(axis=1)[i*stride:i*stride+1]], axis=1).rolling(20).mean()
    #     pred_and_gt.plot(ax=axes[i//4, i%4], legend=True)
    #     axes[i//4, i%4].set_title(label.columns[i*stride])
    # plt.title(f"{dataset} {model} {seq_length}")
    # plt.savefig(rf"/home/xuyang1/PatchTST/PatchTST_supervised/plot_result/{dataset}_{model}_{seq_length}.png")
    # plt.show()
    
    # loss = ((real_pred - target_label)**2).mean(axis=0)
    # losses.append(loss.to_frame(f"{dataset}_{model}_{seq_length}").T)
    # loss = (np.quantile((real_pred - target_label).values.flatten(), 0.95))
    # error = (real_pred - target_label).values.flatten()
    # hist, bin_edges = np.histogram(error, bins=600, range=(-3, 3))
    # if "quintile" in dataset and seq_length == "96":
    #     plt.plot(bin_edges[:-1], hist, label=f"{dataset} {model} {seq_length}")
    # rics = []
    # for col in pred_df.columns:
    #     rics.append(pred_df[col].corr(target_label[col], method="spearman"))

    # error = (real_pred - target_label).mean(axis=1).to_frame(f"l1 loss {dataset} {seq_length} {model}")
    # losses.append(error)
    # error = ((real_pred - target_label)**2).mean(axis=1).to_frame(f"l2 loss {dataset} {seq_length} {model}")
    # losses.append(error)

    # print(dataset, seq_length, model, sum(rics) / len(rics), sep=",")
# loss_df = pd.concat(losses, axis=1)
# print(loss_df.shape)

In [ ]:

loss_df = loss_df.T.sort_index().T
# display(loss_df.head())
fig = px.line(loss_df.rolling(20).mean(), title="rolling 20 error")
fig.write_html("/home/xuyang1/PatchTST/PatchTST_supervised/plot_result/rolling_20_error.html")
fig.show()

## 处理美股数据


In [ ]:
import pandas as pd
from tqdm import tqdm 
from functools import partial
import pickle


In [ ]:

msn_data = pd.read_parquet(r"/home/xuyang1/PatchTST/es_data/msn_data.parq")
price = pd.read_parquet(r"/home/xuyang1/PatchTST/es_data/orig_price_info.parquet")

msn_data = msn_data.iloc(axis=1)[:-1]
msn_data.columns = [i[1] for i in msn_data.columns]
def guess_price(df, shift, horizon):
    return df.price.shift(-shift-horizon) / df.price.shift(-shift) - 1

tqdm.pandas()
price["price_1d"] = price.groupby("instrument", group_keys=False).progress_apply(partial(guess_price, shift=1, horizon=1)).sort_index()
new_data = msn_data.join(price["price_1d"], how="left").dropna(subset=['price_1d'])

In [ ]:
from pandarallel import pandarallel
pandarallel.initialize()

tqdm.pandas()
def calculate_quintile_ret(df):
    label = df.iloc(axis=1)[-1]
    quintile_ret = pd.Series(index=df.columns)
    for col in df.columns:
        tmp_df = pd.DataFrame({"left": df[col], "right": label}).sort_values("left", ascending=False)
        group1_ret = tmp_df.iloc[:int(len(tmp_df) * 0.2), 1].mean()
        group5_ret = tmp_df.iloc[int(len(tmp_df) * 0.8):, 1].mean()
        quintile_ret[col] = group1_ret - group5_ret

    return quintile_ret

quintile_ret_data = new_data.groupby("datetime").parallel_apply(calculate_quintile_ret)
pickle.dump(quintile_ret_data, open(r'/home/xuyang1/PatchTST/es_data/quintile_ret_data.pkl', 'wb'))

In [ ]:
from pandarallel import pandarallel
pandarallel.initialize()

def calculate_top10p_ret(df):
    label = df.iloc(axis=1)[-1]
    top10p_ret = pd.Series(index=df.columns)
    for col in df.columns:
        tmp_df = pd.DataFrame({"left": df[col], "right": label}).sort_values("left", ascending=False)
        top10p_ret[col] = tmp_df.iloc[:int(len(tmp_df) * 0.1), 1].mean()
    return top10p_ret

top10p_ret_data = new_data.groupby("datetime").parallel_apply(calculate_top10p_ret)
pickle.dump(top10p_ret_data, open(r'/home/xuyang1/PatchTST/es_data/top10p_ret_data.pkl', 'wb'))

In [ ]:
quintile_ret_data = pickle.load(open(r'/home/xuyang1/PatchTST/es_data/quintile_ret_data.pkl', 'rb'))
top10p_ret_data = pickle.load(open(r'/home/xuyang1/PatchTST/es_data/top10p_ret_data.pkl', 'rb'))


In [ ]:
for col in tqdm(quintile_ret_data.columns):
    count = 0
    while True:
        upper = quintile_ret_data[col].mean() + quintile_ret_data[col].std() * 5
        lower = quintile_ret_data[col].mean() - quintile_ret_data[col].std() * 5
        if (upper > quintile_ret_data[col].max()) and (lower < quintile_ret_data[col].min()):
            break
        if count > 3:
            break
        count += 1
        quintile_ret_data[col] = quintile_ret_data[col].clip(lower, upper,)

for col in tqdm(top10p_ret_data.columns):
    count = 0
    while True:
        upper = top10p_ret_data[col].mean() + top10p_ret_data[col].std() * 5
        lower = top10p_ret_data[col].mean() - top10p_ret_data[col].std() * 5
        if (upper > top10p_ret_data[col].max()) and (lower < top10p_ret_data[col].min()):
            break
        if count > 3:
            break
        count += 1
        top10p_ret_data[col] = top10p_ret_data[col].clip(lower, upper,)

In [ ]:
quintile_ret_data = quintile_ret_data.iloc(axis=1)[:-1]
top10p_ret_data = top10p_ret_data.iloc(axis=1)[:-1]

quintile_ret_data.index.names = ["date"]
top10p_ret_data.index.names = ["date"]

quintile_ret_data.to_csv(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_us.csv')
top10p_ret_data.to_csv(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_us.csv')


## 验证美股结果

In [ ]:
import pandas as pd
import os
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

label_quintile = pd.read_csv(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_us.csv', index_col=0)
label_quintile = pd.DataFrame(pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_us_transformed.pkl', 'rb')), index=label_quintile.index, columns=label_quintile.columns)
# quintile_scaler = pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/quintile_us_scaler.pkl', 'rb'))
label_top10p = pd.read_csv(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_us.csv', index_col=0)
label_top10p = pd.DataFrame(pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_us_transformed.pkl', 'rb')), index=label_quintile.index, columns=label_quintile.columns)
# top10p_scaler = pickle.load(open(r'/home/xuyang1/PatchTST/PatchTST_supervised/dataset/top10p_us_scaler.pkl', 'rb'))
losses = []
for name in os.listdir(r"/home/xuyang1/PatchTST/PatchTST_supervised/results"):
    if "stocka" in name:
        continue
    model = name.split("_")[4]
    seq_length = name.split("_")[3]
    dataset = name.split("_")[0]
    if dataset == "quintile":
        label = label_quintile
    elif dataset == "top10p":
        label = label_top10p
    pred = np.load(rf"/home/xuyang1/PatchTST/PatchTST_supervised/results/{name}/pred.npy")

    # if dataset == "quintile":
    #     scaler = quintile_scaler
    # elif dataset == "top10p":
    #     scaler = top10p_scaler
    # pred = scaler.inverse_transform(np.reshape(pred, (-1, pred.shape[-1]))).reshape(pred.shape)

    # if "quinti" not in dataset or seq_length != "96":
    #     continue

    test_length = pred.shape[0] + pred.shape[1] - 1
    target_label = label.iloc[-test_length:,:]


    # calculate pred to real label loss:
    # loss = 0
    # for i in range(pred.shape[0]):
    #     pred_batch = pred[i]
    #     label_batch = target_label.iloc[i:i+pred.shape[1],:].values
    #     loss += np.mean((pred_batch - label_batch)**2)

    # calculate real return with prediction:
    # 1. use closest pred as real pread
    # real_pred = np.concatenate([pred[:-1,0,:], pred[-1,:,:]], axis=0)

    # 2. use mean of pred as real pred
    # pred_middata_np = np.zeros((pred.shape[0], pred.shape[0] + pred.shape[1] - 1, pred.shape[2]))
    # pred_middata_np[:,:,:] = np.nan
    # for i in range(pred.shape[0]):
    #     pred_middata_np[i, i:i+pred.shape[1], :] = pred[i]
    # real_pred = np.nanmean(pred_middata_np, axis=0, keepdims=False)


    # 3. use mean of pred as real pred, but only use the first 20 pred
    # real_pred = pred[:, :20, :].mean(axis=1, keepdims=False)

    # 4. use farest pred as real pred
    real_pred = np.concatenate([pred[0,:,:], pred[1:,-1,:]], axis=0)

    pred_df = pd.DataFrame(real_pred, index=target_label.index[:real_pred.shape[0]], columns=target_label.columns)

    # rolling_20_label = target_label.rolling(20).sum().shift(-20).iloc(axis=0)[:pred.shape[0]]

    # average_ret = rolling_20_label.mean(axis=1).to_frame("average_ret")

    # rolling_20_pred_df = pred_df.rolling(20).mean().shift(-20).iloc(axis=0)[:-20]
    # top50p_pred_ret = rolling_20_label[(pred_df.rank(axis=1, pct=True) > 0.5)].mean(axis=1).to_frame("top50p_pred_ret")
    # top20p_pred_ret = rolling_20_label[(pred_df.rank(axis=1, pct=True) > 0.8)].mean(axis=1).to_frame("top20p_pred_ret")
    # top10p_pred_ret = rolling_20_label[(pred_df.rank(axis=1, pct=True) > 0.9)].mean(axis=1).to_frame("top10p_pred_ret")
    # break

    # show_df = pd.concat([average_ret, top50p_pred_ret, top20p_pred_ret, top10p_pred_ret], axis=1).cumsum(axis=0)
    # show_df.plot()
    # plt.title(f"{dataset} {model} {seq_length}")
    # plt.show()

    pred_with_input = pd.concat([label.iloc[:-test_length,:], pred_df], axis=0)
    pred_with_input.columns = [col + "_pred" for col in pred_with_input.columns]

    fig, axes = plt.subplots(5, 5)
    fig.set_figheight(30)
    fig.set_figwidth(30)
    plt.subplots_adjust(wspace=None, hspace=None)
    axes = axes.reshape(5, 5)
    stride = label.shape[-1] // 25
    for i in range(25):
        pred_and_gt = pd.concat([label.iloc(axis=1)[i*stride:i*stride+1], pred_with_input.iloc(axis=1)[i*stride:i*stride+1]], axis=1).cumsum(axis=0)
        pred_and_gt.plot(ax=axes[i//5, i%5], legend=True)
        axes[i//5, i%5].set_title(label.columns[i*stride])
    plt.title(f"{dataset} {model} {seq_length}")
    plt.savefig(rf"/home/xuyang1/PatchTST/PatchTST_supervised/plot_result/{dataset}_{model}_{seq_length}.png")
    # plt.show()
    
    # loss = ((real_pred - target_label)**2).mean(axis=0)
    # losses.append(loss.to_frame(f"{dataset}_{model}_{seq_length}").T)
    # loss = (np.quantile((real_pred - target_label).values.flatten(), 0.95))
    # error = (real_pred - target_label).values.flatten()
    # hist, bin_edges = np.histogram(error, bins=600, range=(-3, 3))
    # if "quintile" in dataset and seq_length == "96":
    #     plt.plot(bin_edges[:-1], hist, label=f"{dataset} {model} {seq_length}")
    # rics = []
    # for col in pred_df.columns:
    #     rics.append(pred_df[col].corr(target_label[col], method="spearman"))

    # error = (real_pred - target_label).mean(axis=1).to_frame(f"l1 loss {dataset} {seq_length} {model}")
    # losses.append(error)
    # error = ((real_pred - target_label)**2).mean(axis=1).to_frame(f"l2 loss {dataset} {seq_length} {model}")
    # losses.append(error)

    # print(dataset, seq_length, model, sum(rics) / len(rics), sep=",")
# loss_df = pd.concat(losses, axis=1)
# print(loss_df.shape)